In [0]:
FIELD = 'ursaminor'
ISOCHRONES_PATH = '/datascope/subaru/data/cmdfit/isochrones/dartmouth/import/afep0_cfht_sdss_hsc'
OBS_FILE = '$PFS_TARGETING_DATA/data/targeting/dSph/ursaminor/ursaminor_tpall3e_g24.cat'
SAMPLE_FILE = '$PFS_TARGETING_DATA/data/targeting/dSph/ursaminor/priority/ursaminor_nb_2/hsc_umi_priorities.feather'
PMAP_FILE = '$PFS_TARGETING_DATA/data/targeting/dSph/ursaminor/pmap/ursaminor_nb/pmap.h5'
ASSIGNMENTS_FILE = '$PFS_TARGETING_DATA/data/targeting/dSph/ursaminor/netflow/SSP/ursaminor_6/umi_assignments_all.feather'

In [0]:
import os, sys
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Ellipse, Circle
from matplotlib.gridspec import GridSpec

In [0]:
plt.rc('font', size=6)

In [0]:
# if 'debug' not in globals():
#     import debugpy
#     debugpy.listen(('0.0.0.0', 5698))
#     debug = True

In [0]:
%load_ext autoreload
%autoreload 2

# Plot definitions

In [0]:
from pfs.ga.targeting.targets.dsph import GALAXIES as DSPH_FIELDS
from pfs.ga.targeting.targets.m31 import M31_FIELDS
from pfs.ga.targeting.instrument import *
from pfs.ga.targeting.diagram import CMD, CCD, FOV, FP, ColorAxis, MagnitudeAxis
from pfs.ga.targeting.photometry import Photometry, Magnitude, Color
from pfs.ga.targeting.projection import Pointing, WcsProjection

In [0]:
field = DSPH_FIELDS[FIELD]
hsc = field.get_photometry()
cmd = field.get_cmd()
ccd = field.get_ccd()

In [0]:
pointing = field.get_center()
pointing

In [0]:
wcs = WcsProjection(pointing, proj='TAN')
wfc = SubaruWFC(pointing)
fov = FOV(projection=wcs)
fp = FP(wfc)

# Load isochrones

In [0]:
from pfs.ga.isochrones.isogrid import IsoGrid

In [0]:
iso = IsoGrid()
iso.load(os.path.join(ISOCHRONES_PATH, 'isochrones.h5'))
iso.values.keys()

# Load observations

In [0]:
from pfs.ga.targeting.io import ObservationSerializer

In [0]:
obs = field.get_text_observation_reader().read(os.path.expandvars(OBS_FILE))
obs.data.shape

In [0]:
sample = ObservationSerializer().read(os.path.expandvars(SAMPLE_FILE))
sample.data.shape

In [0]:
cmap = plt.get_cmap('tab10')

f = plt.figure(figsize=(8, 3), dpi=240)
gs = f.add_gridspec(1, 3, width_ratios=[2, 2, 3], wspace=0.4)   

ax = f.add_subplot(gs[0])
cmd.plot_observation(ax, obs, c='lightgray')
cmd.plot_observation(ax, sample, c='r', cmap=cmap)

ax = f.add_subplot(gs[1])
ccd.plot_observation(ax, obs, c='lightgray')
ccd.plot_observation(ax, sample, c='r', cmap=cmap)

ax = f.add_subplot(gs[2], projection=wcs.wcs)
fov.plot_observation(ax, obs, c='lightgray')
fov.plot_observation(ax, sample, c='r', cmap=cmap)

# f.tight_layout()

In [0]:
cmap = plt.get_cmap('tab10')

f = plt.figure(figsize=(8, 3), dpi=120)
gs = f.add_gridspec(1, 3, width_ratios=[2, 2, 3], wspace=0.4)   

ax = f.add_subplot(gs[0])
cmd.plot_observation(ax, obs, c='lightgray')
cmd.plot_observation(ax, sample, c=sample.data['priority'], cmap=cmap)

ax = f.add_subplot(gs[1])
ccd.plot_observation(ax, obs, c='lightgray')
ccd.plot_observation(ax, sample, c=sample.data['priority'], cmap=cmap)

ax = f.add_subplot(gs[2], projection=wcs.wcs)
fov.plot_observation(ax, obs, c='lightgray')
l = fov.plot_observation(ax, sample, c=sample.data['priority'], cmap=cmap)
ax.set_aspect('auto')

f.colorbar(l)

In [0]:
mask = ~np.isnan(sample.data['priority'])
priorities = list(sorted(sample.data['priority'][mask].unique()))

for i, pri in enumerate(priorities):
    mask = sample.data['priority'] == pri

    f = plt.figure(figsize=(8, 3), dpi=120)
    gs = f.add_gridspec(1, 3, width_ratios=[2, 2, 3], wspace=0.4)   

    ax = f.add_subplot(gs[0])
    cmd.plot_observation(ax, obs, c='lightgray')
    cmd.plot_observation(ax, sample, c='red', mask=mask)

    ax = f.add_subplot(gs[1])
    ccd.plot_observation(ax, obs, c='lightgray')
    ccd.plot_observation(ax, sample, c='red', mask=mask)
    ax.set_title(f'PRIORITY = {pri}')

    ax = f.add_subplot(gs[2], projection=wcs.wcs)
    fov.plot_observation(ax, obs, c='lightgray')
    fov.plot_observation(ax, sample, c='red', mask=mask)

# Load probability map

In [0]:
from pfs.ga.targeting import ProbabilityMap
from pfs.ga.targeting.selection import ProbabilityCut, ProbabilitySampling

In [0]:
pmap = ProbabilityMap(cmd.axes)
pmap.load(os.path.expandvars(PMAP_FILE))

In [0]:
f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120)

l0 = cmd.plot_probability_map(axs[0], pmap, 0)
axs[0].set_title("non-member")

l1 = cmd.plot_probability_map(axs[1], pmap, 1)
axs[1].set_title("member")

f.tight_layout()

In [0]:
probcut = ProbabilityCut(pmap, 1, np.log(0.001))
probcut

## Ghost plots

In [0]:
# mask = get_selection_mask(obs, nb=True, blue=False, probcut=probcut)
ghost_mask = pmap.create_random_mask(sample)

ghost_mask.shape

In [0]:
f = plt.figure(figsize=(4, 4), dpi=240)
gs = f.add_gridspec(2, 2, width_ratios=[1, 1], height_ratios=[1, 1], wspace=0.35, hspace=0.35)

ax = f.add_subplot(gs[0, 0])
cmd.plot_observation(ax, sample)

ax = f.add_subplot(gs[0, 1])
cmd.plot_observation(ax, sample, mask=~ghost_mask[..., 1])

ax = f.add_subplot(gs[1, 0], projection=wcs.wcs)
fov.plot_observation(ax, sample)

ax = f.add_subplot(gs[1, 1], projection=wcs.wcs)
fov.plot_observation(ax, sample, mask=~ghost_mask[..., 1])
